# Example - usALEX histogram

*This notebook is part of smFRET burst analysis software [FRETBursts](http://tritemio.github.io/FRETBursts/).*

> In this notebook shows how to plot different styles of us-ALEX histograms and $E$ and $S$ marginal distributions.
> For a complete tutorial on burst analysis see 
> [FRETBursts - us-ALEX smFRET burst analysis](FRETBursts - us-ALEX smFRET burst analysis.ipynb).

# Load software

## FRETBursts

In [ ]:
from fretbursts import *

In [ ]:
sns = init_notebook()

## Interactive widgets

In [ ]:
from IPython.html.widgets import interact, interactive, fixed
from IPython.html import widgets
from IPython.display import display, display_png, display_svg, clear_output
from IPython.core.pylabtools import print_figure

# Download the sample file

In [ ]:
url = 'http://files.figshare.com/2182601/0023uLRpitc_NTP_20dT_0.5GndCl.hdf5'
download_file(url, save_dir='./data')

# Select a data file

In [ ]:
file_name = "0023uLRpitc_NTP_20dT_0.5GndCl.hdf5"

# Here the folder is the subfolder "data" of current notebook folder
folder_name = './data/'
full_fname = folder_name + file_name
full_fname

Let's check that the file exists:

In [ ]:
import os
if os.path.isfile(full_fname):
    print("Perfect, I found the file!")
else:
    print("Sorry, I can't find the file:\n", full_fname)

# Load the selected file

In [ ]:
d = loader.photon_hdf5(full_fname)
#d.add(det_donor_accept=(0, 1), alex_period=4000, 
#      offset=700, D_ON=(2180, 3900), A_ON=(200, 1800))
bpl.plot_alternation_hist(d)
loader.alex_apply_period(d)
d.calc_bg(bg.exp_fit, time_s=1000, tail_min_us=(800, 4000, 1500, 1000, 3000))

# Burst search

In [ ]:
d.burst_search(L=10, m=10, F=6)
ds = Sel(d, select_bursts.size, add_naa=True, th1=30)

In [ ]:
#dplot(ds, hist_fret)
#dplot(ds, hist_S);

In [ ]:
#ds.E_fitter.fit_histogram(mfit.factory_three_gaussians())

# ALEX plots

## Basics

We can make a simple E-S scatter plot with `scatter_alex`:

In [ ]:
dplot(ds, scatter_alex, figsize=(4,4), mew=1, ms=4, mec='black', color='purple');

We can also plot the ALEX histogram with a scatterplot overlay using `hist2d_alex`:

In [ ]:
dplot(ds, hist2d_alex);

# Colormaps

Here we plot a small selection of [Matplotlib's colormaps](http://matplotlib.org/users/colormaps.html):

In [ ]:
cmaps = ['viridis', 'plasma', 'inferno', 'magma',
         'afmhot', 'Blues', 'BuGn', 'BuPu', 'GnBu', 'YlGnBu',
         'coolwarm', 'RdYlBu', 'RdYlGn', 'Spectral']

In [ ]:
def plot_color_gradients(cmap_list):
    gradient = np.linspace(0, 1, 256)
    gradient = np.vstack((gradient, gradient))
    fig, axes = plt.subplots(nrows=len(cmap_list))
    fig.subplots_adjust(top=0.95, bottom=0.01, left=0.2, right=0.99)
    for ax, name in zip(axes, cmap_list):
        ax.imshow(gradient, aspect='auto', cmap=plt.get_cmap(name))
        pos = list(ax.get_position().bounds)
        x_text = pos[0] - 0.01
        y_text = pos[1] + pos[3]/2.
        fig.text(x_text, y_text, name, va='center', ha='right')
    for ax in axes:
        ax.set_axis_off()

In [ ]:
plot_color_gradients(cmaps)

# ALEX Joint Plot

In [ ]:
sns.set_style('darkgrid')

In [ ]:
# Fit E and S to a model and compute KDE
bext.bursts_fitter(ds, 'E', binwidth=0.03, bandwidth=0.03, model=mfit.factory_three_gaussians())
bext.bursts_fitter(ds, 'S', binwidth=0.03, bandwidth=0.03, model=mfit.factory_two_gaussians())

In [ ]:
# Limiting the range of E and S works better for 2D KDE plots
dss = ds.select_bursts(select_bursts.ES, E1=-0.2, E2=1.2, S1=-0.2, S2=1.2)

In [ ]:
# Remove D-only and A-only through burst selection
dssf = dss.select_bursts(select_bursts.size, th1=20)
dssf = dssf.select_bursts(select_bursts.naa, th1=20)

Just a test plot:

In [ ]:
E, S = dss.E_, dss.S_
fig, ax = plt.subplots(figsize=(6, 6))
sns.kdeplot(E, S, shade=True, n_levels=30, cmap='alex_dark', shade_lowest=False)
plt.xlim(-0.19, 1.19)
plt.ylim(-0.19, 1.19)

In [ ]:
plt.imshow(np.arange(100).reshape(10, 10), cmap='alex_dark')

## Joint plots

Here we show several jointplots. 
The argument `joint_kws` is reported with typical keywords
used to customize the 2-D (inner) plot.
FRETBursts defines 'alex_light' and 'alex_dark' colormaps in addition to
the matplotlib and seaborn ones. 

You can inver matplotlib colormaps appending `_r` to the name. For the full list see [matplotlib colormaps](http://matplotlib.org/users/colormaps.html).

In [ ]:
alex_jointplot(ds, kind='scatter', cmap='Blues_r',
               joint_kws=dict(s=40, alpha=0.1, linewidths=0))

In [ ]:
alex_jointplot(dss, kind='kde', cmap='viridis',
               joint_kws=dict(shade=True, shade_lowest=False, n_levels=30))

In [ ]:
alex_jointplot(ds, kind='hex', cmap='alex_dark',
               joint_kws=dict(edgecolor='grey', linewidth=0.2, gridsize=50))

In [ ]:
alex_jointplot(ds, kind='hex', cmap='inferno',
               joint_kws=dict(edgecolor='none', linewidth=0.001, gridsize=50))

In [ ]:
alex_jointplot(dss, kind='kde', cmap='RdYlBu_r')

In [ ]:
alex_jointplot(dssf, kind='kde', cmap='inferno',
               joint_kws=dict(shade=True, shade_lowest=False, n_levels=20))

In [ ]:
alex_jointplot(ds, joint_kws = dict(edgecolor='gray'))

alex_jointplot(ds, joint_kws = {})

In [ ]:
alex_jointplot(ds)
#plt.savefig('ALEX_jointplot.png', dpi=160, bbox_inches='tight')

In [ ]:
alex_jointplot(ds, cmap='inferno')

In [ ]:
sns.set_style('whitegrid')

In [ ]:
alex_jointplot(ds)

## Interactive plot

In [ ]:
sns.set_style('darkgrid')

In [ ]:
@interact(cmap_compensate = False,
          overlay = widgets.RadioButtons(values=['fit model', 'KDE']),
          binwidth = widgets.FloatText(value=0.03, min=0.01, max=1),
          bandwidth = widgets.FloatText(value=0.03, min=0.01, max=1),
          gridsize = (10, 100),
          min_size=(10, 500, 5),
          cmap=widgets.Dropdown(value='RdYlBu', options=cmaps),
          reverse_cmap = True,
          vmax_fret = True,
          )
def plot_(min_size=50, cmap_compensate=False, overlay='KDE', binwidth=0.03, bandwidth=0.03, gridsize=50, cmap='YlGnBu_crop', 
          reverse_cmap=False, vmax_fret=True):
    dx = Sel(d, select_bursts.size, add_naa=True, th1=min_size)
    bext.bursts_fitter(dx, 'E', binwidth=binwidth, bandwidth=bandwidth, 
                       model=mfit.factory_three_gaussians())
    bext.bursts_fitter(dx, 'S', binwidth=binwidth, bandwidth=bandwidth, 
                       model=mfit.factory_two_gaussians())    
    
    if reverse_cmap: cmap += '_r'

    if binwidth < 0.01: binwidth = 0.01
    if bandwidth < 0.01: bandwidth = 0.01
    if overlay == 'fit model':
        marginal_kws = dict(binwidth=binwidth, show_model=True)
    else:
        marginal_kws = dict(binwidth=binwidth, show_kde=True, bandwidth=bandwidth)
    histcolor = sns.color_palette(cmap, 12)[1]
    marginal_kws['hist_bar_style'] = dict(facecolor=histcolor, edgecolor='k', linewidth=0.2)  
    alex_jointplot(dx, cmap=cmap, gridsize=gridsize, vmax_fret=vmax_fret, 
                   cmap_compensate=cmap_compensate, 
                   marginal_kws=marginal_kws, 
                   joint_kws=dict(edgecolor='grey', lw=0.2))
    
    fig = gcf()
    plt.close()
    display(fig)